# First eval

This jupyter notebook is for trying out what research I have read and my ideas in this [Notion](https://www.notion.so/Model-evaluation-170d3cfe027d80889de3cfbb35b531a4).

I played around with a Mistral Large model to build a mathematics chatbot. It is simple and I prompt engineered it. I might change the model to a smaller and cheaper one, but I still want to use a model that is considerd good enough. My plan is to use three small open source models to evaluate the mahtematics model. I'll also use different models for the mathematics chatbot and for the evaluation models to get realible results.

I'll use this [dataset](https://github.com/google-deepmind/mathematics_dataset?tab=readme-ov-file) as a test data set. I'll select 50 mathematics questiosn from the dataset. Then I'll ask the mathematicsbot the same questions. 
Then I'll let: 
- First a Mistral Large model compare the answers. Same model as in the mathematicsbot but different prompt engineering. 
    - This is not a good way to evaluate models, but I wish to try it. Espeacilly when it is only 50 questions it is manageble for me to compare the answers.
- Then I'll let three smaller open-source models compare the answers.
- After that I'll add a fourth model to compare the answers of the three open-open source model to give a final evaluation about the mathematics chat bot.

In [61]:
!pip3 install mistralai
!pip3 install python-dotenv
!pip3 install mathematics_dataset
!pip3 install pip install sympy==1.6
!pip3 install numpy==1.24.3
!pip3 install transformers==4.48.0
!pip3 install tiktoken==0.8.0
!pip3 install protobuf==5.29.3
!pip3 install sentencepiece==0.2.0
!pip3 install torch


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


I've used the following prompt for the Mistral Large 2.1 model
"Your task is to help solve math problems. Generate three answers that helps solving the problem but doesn't give the solution. When generating the 4th answer to the problem you can give the solution. When you don't know the answer say that you don't know the answer. Write the mathematical part of the answer in latex."

In [10]:
import os
from dotenv import load_dotenv
from mistralai import Mistral

load_dotenv()

api_key = os.environ["MISTRAL_API_KEY"]
agent_id = os.environ["MISTRAL_AGENT_ID"]

client = Mistral(api_key=api_key)

chat_response = client.agents.complete(
    agent_id= agent_id,
    messages=[
        {
            "role": "user",
            "content": "Solve 0 = 4*a + 19 - 7 for a."
        },
    ],
)
print(chat_response.choices[0].message.content)

1. First, simplify the equation by combining like terms on the right side.
   \[
   0 = 4a + 12
   \]
   The answer is not final yet.

2. To isolate the term with \( a \), subtract 12 from both sides of the equation.
   \[
   -12 = 4a
   \]
   The answer is not final yet.

3. Divide both sides by 4 to solve for \( a \).
   \[
   a = \frac{-12}{4}
   \]
   The answer is not final yet.

4. Simplify the right side to find the value of \( a \).
   \[
   a = -3
   \]
   The answer is -3


In [ ]:
import os
import json
from dotenv import load_dotenv
from mistralai import Mistral

# Load environment variables
load_dotenv()

# Get API key and agent ID
api_key = os.environ["MISTRAL_API_KEY"]
agent_id = os.environ["MISTRAL_AGENT_ID"]

# Initialize Mistral client
client = Mistral(api_key=api_key)

# Load questions from the JSON file
input_file = "questions.json"  # Replace with your input file name
output_file = "model_answers.json"  # Output file name

with open(input_file, "r") as f:
    questions = json.load(f)

# Initialize a list to store the results
results = []

# Iterate over the questions
for question in questions:
    try:
        # Ask the question to the model
        chat_response = client.agents.complete(
            agent_id=agent_id,
            messages=[
                {
                    "role": "user",
                    "content": question
                },
            ],
        )
        # Extract the model's answer
        answer = chat_response.choices[0].message.content.strip()

        # Append the question and answer to the results
        results.append({
            "question": question,
            "answer": answer
        })

        print(f"Question: {question}")
        print(f"Answer: {answer}\n")
    except Exception as e:
        print(f"Error processing question: {question}")
        print(f"Error: {e}")

# Save the results to the output file
with open(output_file, "w") as f:
    json.dump(results, f, indent=4)

print(f"Results saved to {output_file}")

I have used the [mathematics_dataset](https://github.com/google-deepmind/mathematics_dataset) from Google DeepMind that will work as a test set. The main issue is that I don't know if the Mistral Large model that is used for the mathematics model have been trained on this data. There is a great chans for that. 

Never the less, I made a json file called questions_and_answers of the [mathematics_dataset](https://github.com/google-deepmind/mathematics_dataset) so that I can easily separet the questions from the answers. Ideally I would run all the questions through the mathematics model, but due to budget questions that won't be possible. I'll choose around 50 questions from the dataset. 

In [151]:
import json
import re

def clean_ansi(text):
    """Remove ANSI color codes from text"""
    ansi_pattern = re.compile(r'\x1B(?:[@-Z\\-_]|\\[[0-?]*[ -/]*[@-~])')
    return ansi_pattern.sub('', text)

def process_qa_data(raw_text, output_file="questions_and_answers.json"):
    """Process raw text into question-answer pairs and save to JSON"""
    qa_pairs = []
    
    # Split into lines and process each line
    lines = raw_text.split('\n') if isinstance(raw_text, str) else raw_text
    
    current_section = None
    current_question = []
    previous_answer = None
    
    for line in lines:
        # Skip empty lines
        if not line.strip():
            continue
            
        # Check if this is a section header
        if line.startswith('\u001b[1m') and '\u001b[92m' not in line:
            current_section = clean_ansi(line.strip())
            current_question = []
            previous_answer = None
            continue
            
        # Skip log messages
        if line.startswith('W') and ']' in line:
            continue
        
        # If line contains an answer (marked by green color code)
        if '\u001b[92m' in line:
            parts = line.split('\u001b[92m')
            
            # Get the question part (if any)
            question_part = parts[0].strip()
            
            # If there's a reset code in the question part, it contains a previous answer
            if '\u001b[0m' in question_part:
                q_parts = question_part.split('\u001b[0m')
                if len(q_parts) > 1 and previous_answer is not None:
                    # Complete the previous answer
                    qa_pairs[-1]["answer"] += ' ' + clean_ansi(q_parts[0].strip())
                    # Update the question part to exclude the previous answer
                    question_part = q_parts[1].strip()
            
            # Add any new question part to current question
            if question_part:
                current_question.append(question_part)
            
            # Get the answer
            answer = parts[1].strip()
            if '\u001b[0m' in answer:
                answer = answer.split('\u001b[0m')[0].strip()
            
            # Combine all question parts
            full_question = ' '.join(current_question).strip()
            
            # Add the new QA pair
            if answer:
                qa_pairs.append({
                    "section": clean_ansi(current_section) if current_section else None,
                    "question": full_question,
                    "answer": clean_ansi(answer)
                })
                previous_answer = answer
            
            # Reset question accumulator
            current_question = []
        else:
            # If line doesn't contain an answer, check if it contains a previous answer completion
            if '\u001b[0m' in line:
                parts = line.split('\u001b[0m')
                if previous_answer is not None:
                    # Add to previous answer
                    qa_pairs[-1]["answer"] += ' ' + clean_ansi(parts[0].strip())
                    # Add remaining part to current question
                    if len(parts) > 1 and parts[1].strip():
                        current_question.append(parts[1].strip())
            else:
                # Regular question part
                current_question.append(line.strip())
    
    # Filter out any entries with empty questions
    qa_pairs = [pair for pair in qa_pairs if pair["answer"] and pair["question"]]
    
    # Save to JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(qa_pairs, f, indent=2)
    
    print(f"Output saved to {output_file}")
    return qa_pairs

# Example usage:
if __name__ == "__main__":
    # Get the output from mathematics_dataset.generate
    output = !python -m mathematics_dataset.generate
    
    # Process the output
    qa_data = process_qa_data(output)

Output saved to questions_and_answers.json


In [152]:
import json

def clean_specific_ansi(text):
    """Remove specific ANSI codes"""
    if text:
        return text.replace('\u001b[1m', '').replace('\u001b[0m', '')
    return text

def clean_json_file(input_file="questions_and_answers.json", output_file="clean_qa_data.json"):
    # Read the JSON file
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Clean specific ANSI codes from each field
    clean_data = []
    for item in data:
        clean_item = {
            "section": clean_specific_ansi(item["section"]),
            "question": clean_specific_ansi(item["question"]),
            "answer": clean_specific_ansi(item["answer"])
        }
        clean_data.append(clean_item)
    
    # Write the cleaned data back to a new JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(clean_data, f, indent=2)
    
    print(f"Cleaned data saved to {output_file}")
    return clean_data

# Run the cleaning
clean_data = clean_json_file()

Cleaned data saved to clean_qa_data.json


In [153]:

# Load the JSON data from the file
with open('clean_qa_data.json', 'r') as json_file:
    data = json.load(json_file)

# Extract only the questions
questions = [entry["question"] for entry in data]

# Optionally, you can save the questions to a new JSON file
with open('questions.json', 'w') as json_file:
    json.dump(questions, json_file, indent=4)

In [18]:
import json
# Sample data to try without extra costs
# Load the JSON data from the file
with open('sample_clean_qa_data.json', 'r') as json_file:
    data = json.load(json_file)

# Extract only the questions
questions = [entry["question"] for entry in data]

# Optionally, you can save the questions to a new JSON file
with open('sample_questions.json', 'w') as json_file:
    json.dump(questions, json_file, indent=4)

## Finding evaluation model

### Google T5 model (failed)

Try "google/t5-small-ssm-nq" cannot classify answers correctly when the Mistral Large model gives the output in the form "The answer is 4" and it compares it to only the mathematical answer "4". 

In [3]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from sympy import sympify, Eq, simplify

# Load the T5 model and tokenizer
model_name = "google/t5-small-ssm-nq"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5TokenizerFast.from_pretrained(model_name)

# Example datasets
dataset = [
    {"question": "What is 2 + 2?", "expected_answer": "4", "mistral_answer": "The answer is 4"},
    {"question": "What is 3 * 3?", "expected_answer": "9", "mistral_answer": "When multiplying, 3 * 3 equals 9"},
    {"question": "What is 10 - 3?", "expected_answer": "7", "mistral_answer": "Subtracting, we get 6"}
]

# Function to validate correctness using the model
def validate_correctness(question, mistral_answer, expected_answer):
    def extract_math(expr):
        """
        Extract and simplify the mathematical part of an expression.
        Returns the simplified math expression or None if parsing fails.
        """
        try:
            return simplify(sympify(expr))
        except:
            return None

    # Use T5 to classify the correctness of the Mistral model's output
    prompt = (
        f"Evaluate if the given answer is correct:\n"
        f"Question: {question}\n"
        f"Mistral's Answer: {mistral_answer}\n"
        f"Expected Answer: {expected_answer}\n"
        f"Respond with 'Correct' or 'Incorrect'."
    )

    # Generate T5's judgment
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=10)
    model_judgment = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Also validate mathematically
    mistral_expr = extract_math(mistral_answer)
    expected_expr = extract_math(expected_answer)
    mathematical_correct = mistral_expr == expected_expr

    return {
        "model_judgment": model_judgment,
        "mathematical_correct": mathematical_correct
    }

# Evaluate the dataset
results = []
for item in dataset:
    result = validate_correctness(
        question=item["question"],
        mistral_answer=item["mistral_answer"],
        expected_answer=item["expected_answer"]
    )
    results.append({**item, **result})

# Print results
for res in results:
    print(res)


{'question': 'What is 2 + 2?', 'expected_answer': '4', 'mistral_answer': 'The answer is 4', 'model_judgment': 'incorrect answers', 'mathematical_correct': False}
{'question': 'What is 3 * 3?', 'expected_answer': '9', 'mistral_answer': 'When multiplying, 3 * 3 equals 9', 'model_judgment': 'incorrect answers', 'mathematical_correct': False}
{'question': 'What is 10 - 3?', 'expected_answer': '7', 'mistral_answer': 'Subtracting, we get 6', 'model_judgment': 'incorrect answers', 'mathematical_correct': False}


On the other hand it can do a direct comparisson. So when it is an exact match it can compare the answers correctly. 

In [4]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from sympy import sympify, Eq, simplify

# Load the T5 model and tokenizer
model_name = "google/t5-small-ssm-nq"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5TokenizerFast.from_pretrained(model_name)

# Example datasets
dataset = [
    {"question": "What is 2 + 2?", "expected_answer": "4", "mistral_answer": "4"},
    {"question": "What is 3 * 3?", "expected_answer": "9", "mistral_answer": "9"},
    {"question": "What is 10 - 3?", "expected_answer": "7", "mistral_answer": "6"}
]

# Function to validate correctness using the model
def validate_correctness(question, mistral_answer, expected_answer):
    def extract_math(expr):
        """
        Extract and simplify the mathematical part of an expression.
        Returns the simplified math expression or None if parsing fails.
        """
        try:
            return simplify(sympify(expr))
        except:
            return None

    # Use T5 to classify the correctness of the Mistral model's output
    prompt = (
        f"Evaluate if the given answer is correct:\n"
        f"Question: {question}\n"
        f"Mistral's Answer: {mistral_answer}\n"
        f"Expected Answer: {expected_answer}\n"
        f"Respond with 'Correct' or 'Incorrect'."
    )

    # Generate T5's judgment
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=10)
    model_judgment = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Also validate mathematically
    mistral_expr = extract_math(mistral_answer)
    expected_expr = extract_math(expected_answer)
    mathematical_correct = mistral_expr == expected_expr

    return {
        "model_judgment": model_judgment,
        "mathematical_correct": mathematical_correct
    }

# Evaluate the dataset
results = []
for item in dataset:
    result = validate_correctness(
        question=item["question"],
        mistral_answer=item["mistral_answer"],
        expected_answer=item["expected_answer"]
    )
    results.append({**item, **result})

# Print results
for res in results:
    print(res)


{'question': 'What is 2 + 2?', 'expected_answer': '4', 'mistral_answer': '4', 'model_judgment': 'incorrect answers', 'mathematical_correct': True}
{'question': 'What is 3 * 3?', 'expected_answer': '9', 'mistral_answer': '9', 'model_judgment': 'correction', 'mathematical_correct': True}
{'question': 'What is 10 - 3?', 'expected_answer': '7', 'mistral_answer': '6', 'model_judgment': 'Mistral', 'mathematical_correct': False}


When you use regulatory expressions to remove symbols, it removes too much or too little so it it not a viable solution either. 

In [5]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from sympy import sympify, simplify
import re

# Load the T5 model and tokenizer
model_name = "google/t5-small-ssm-nq"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5TokenizerFast.from_pretrained(model_name)

# Example datasets
dataset = [
    {"question": "What is 2 + 2?", "expected_answer": "4", "mistral_answer": "The answer is 4"},
    {"question": "What is 3 * 3?", "expected_answer": "9", "mistral_answer": "When multiplying, 3 * 3 equals 9"},
    {"question": "What is 10 - 3?", "expected_answer": "7", "mistral_answer": "Subtracting, we get 6"}
]

# Function to extract and simplify mathematical expression
def extract_math(expr):
    """
    Extracts and simplifies mathematical expressions from strings,
    removing all non-mathematical text and focusing on numbers.
    """
    # Remove any non-numeric characters except for arithmetic symbols
    expr = re.sub(r'[^\d\+\-\*/\.^\(\)]+', '', expr)
    try:
        # Return the simplified expression
        return simplify(sympify(expr))
    except:
        return None

# Function to validate correctness using the model
def validate_correctness(question, mistral_answer, expected_answer):
    # Use T5 to classify the correctness of the Mistral model's output
    prompt = (
        f"Evaluate if the given answer is correct:\n"
        f"Question: {question}\n"
        f"Mistral's Answer: {mistral_answer}\n"
        f"Expected Answer: {expected_answer}\n"
        f"Respond with 'Correct' or 'Incorrect'."
    )

    # Generate T5's judgment
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=10)
    model_judgment = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Also validate mathematically
    mistral_expr = extract_math(mistral_answer)
    expected_expr = extract_math(expected_answer)
    mathematical_correct = mistral_expr == expected_expr

    return {
        "model_judgment": model_judgment,
        "mathematical_correct": mathematical_correct
    }

# Evaluate the dataset
results = []
for item in dataset:
    result = validate_correctness(
        question=item["question"],
        mistral_answer=item["mistral_answer"],
        expected_answer=item["expected_answer"]
    )
    results.append({**item, **result})

# Print results
for res in results:
    print(res)


{'question': 'What is 2 + 2?', 'expected_answer': '4', 'mistral_answer': 'The answer is 4', 'model_judgment': 'incorrect answers', 'mathematical_correct': True}
{'question': 'What is 3 * 3?', 'expected_answer': '9', 'mistral_answer': 'When multiplying, 3 * 3 equals 9', 'model_judgment': 'incorrect answers', 'mathematical_correct': False}
{'question': 'What is 10 - 3?', 'expected_answer': '7', 'mistral_answer': 'Subtracting, we get 6', 'model_judgment': 'incorrect answers', 'mathematical_correct': False}


Prompt engineering doesn't work either.

In [7]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast

# Load the T5 model and tokenizer
model_name = "google/t5-small-ssm-nq"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5TokenizerFast.from_pretrained(model_name)

# Example dataset
dataset = [
    {"question": "What is 2 + 2?", "expected_answer": "4", "mistral_answer": "The answer is 4"},
    {"question": "What is 3 * 3?", "expected_answer": "9", "mistral_answer": "When multiplying, 3 * 3 equals 9"},
    {"question": "What is 10 - 3?", "expected_answer": "7", "mistral_answer": "Subtracting, we get 6"}
]

# Function to validate correctness using the model
def validate_correctness(question, mistral_answer, expected_answer):
    # Construct the prompt to guide the model to compare only the mathematical part
    prompt = (
        f"Question: {question}\n"
        f"Expected Answer: {expected_answer}\n"
        f"Mistral's Answer: {mistral_answer}\n"
        f"Check if the provided answer is correct. Only provide 'Correct' or 'Incorrect' based on the numerical correctness of the final answer."
    )

    # Tokenize the prompt and generate the model's response
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=10)
    model_judgment = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Return the model's judgment
    return {
        "model_judgment": model_judgment
    }

# Evaluate the dataset
results = []
for item in dataset:
    result = validate_correctness(
        question=item["question"],
        mistral_answer=item["mistral_answer"],
        expected_answer=item["expected_answer"]
    )
    results.append({**item, **result})

# Print results
for res in results:
    print(res)


{'question': 'What is 2 + 2?', 'expected_answer': '4', 'mistral_answer': 'The answer is 4', 'model_judgment': 'incorrect answers'}
{'question': 'What is 3 * 3?', 'expected_answer': '9', 'mistral_answer': 'When multiplying, 3 * 3 equals 9', 'model_judgment': 'sets a real number'}
{'question': 'What is 10 - 3?', 'expected_answer': '7', 'mistral_answer': 'Subtracting, we get 6', 'model_judgment': 'sets the answers'}


### EleutherAI/gpt-neo-2.7B

Changed model and worked on the prompt, but it wasn't able to correctly compare answers.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the evaluation model and tokenizer
model_name = "EleutherAI/gpt-neo-2.7B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example dataset
dataset = [
    {"question": "What is 2 + 2?", "expected_answer": "4", "mistral_answer": "The answer is 4"},
    {"question": "What is 3 * 3?", "expected_answer": "9", "mistral_answer": "When multiplying, 3 * 3 equals 9"},
    {"question": "What is 10 - 3?", "expected_answer": "7", "mistral_answer": "Subtracting, we get 6"}
]

# Function to evaluate correctness
def evaluate_correctness(question, mistral_answer, expected_answer):
    # Construct the prompt for numeric comparison
    prompt = (
        f"Question: {question}\n"
        f"Expected answer: {expected_answer}\n"
        f"Answer provided by Mistral: {mistral_answer}\n\n"
        "Compare only the numeric value of the provided answer to the expected answer.\n"
        "Does the number in the provided answer exactly match the expected number?"
        "Respond with 'Yes' or 'No' only."
    )
    
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    
    # Generate the output
    outputs = model.generate(
        inputs.input_ids, 
        max_new_tokens=10,  # Generate only up to 10 new tokens for concise output
        num_return_sequences=1
    )
    
    # Decode the model's response
    model_response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
    
    # Post-process the response to check for "yes" or "no"
    mathematical_correct = "yes" in model_response
    
    return {
        "model_judgment": model_response,
        "mathematical_correct": mathematical_correct
    }

# Evaluate the dataset
results = []
for item in dataset:
    result = evaluate_correctness(
        question=item["question"],
        mistral_answer=item["mistral_answer"],
        expected_answer=item["expected_answer"]
    )
    results.append({**item, **result})

# Print the results
for res in results:
    print(res)


/Users/minettebrink/Documents/evaluation/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end g

{'question': 'What is 2 + 2?', 'expected_answer': '4', 'mistral_answer': 'The answer is 4', 'model_judgment': "question: what is 2 + 2?\nexpected answer: 4\nanswer provided by mistral: the answer is 4\n\ncompare only the numeric value of the provided answer to the expected answer.\ndoes the number in the provided answer exactly match the expected number?respond with 'yes' or 'no' only.\n\nquestion: what is 2 + 2?", 'mathematical_correct': True}
{'question': 'What is 3 * 3?', 'expected_answer': '9', 'mistral_answer': 'When multiplying, 3 * 3 equals 9', 'model_judgment': "question: what is 3 * 3?\nexpected answer: 9\nanswer provided by mistral: when multiplying, 3 * 3 equals 9\n\ncompare only the numeric value of the provided answer to the expected answer.\ndoes the number in the provided answer exactly match the expected number?respond with 'yes' or 'no' only.\n\nquestion: what is 3 * 3?", 'mathematical_correct': True}
{'question': 'What is 10 - 3?', 'expected_answer': '7', 'mistral_ans

Removed the extra text in the answer to see if it would help. It didn't.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the evaluation model and tokenizer
model_name = "EleutherAI/gpt-neo-2.7B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example dataset
dataset = [
    {"question": "What is 2 + 2?", "expected_answer": "4", "mistral_answer": "4"},
    {"question": "What is 3 * 3?", "expected_answer": "9", "mistral_answer": "9"},
    {"question": "What is 10 - 3?", "expected_answer": "7", "mistral_answer": "6"}
]

# Function to evaluate correctness
def evaluate_correctness(question, mistral_answer, expected_answer):
    # Construct the prompt for numeric comparison
    prompt = (
        f"Question: {question}\n"
        f"Expected answer: {expected_answer}\n"
        f"Answer provided by Mistral: {mistral_answer}\n\n"
        "Compare only the numeric value of the provided answer to the expected answer.\n"
        "Does the number in the provided answer exactly match the expected number?"
        "Respond with 'Yes' or 'No' only."
    )
    
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    
    # Generate the output
    outputs = model.generate(
        inputs.input_ids, 
        max_new_tokens=10,  # Generate only up to 10 new tokens for concise output
        num_return_sequences=1
    )
    
    # Decode the model's response
    model_response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
    
    # Post-process the response to check for "yes" or "no"
    mathematical_correct = "yes" in model_response
    
    return {
        "model_judgment": model_response,
        "mathematical_correct": mathematical_correct
    }

# Evaluate the dataset
results = []
for item in dataset:
    result = evaluate_correctness(
        question=item["question"],
        mistral_answer=item["mistral_answer"],
        expected_answer=item["expected_answer"]
    )
    results.append({**item, **result})

# Print the results
for res in results:
    print(res)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'question': 'What is 2 + 2?', 'expected_answer': '4', 'mistral_answer': '4', 'model_judgment': "question: what is 2 + 2?\nexpected answer: 4\nanswer provided by mistral: 4\n\ncompare only the numeric value of the provided answer to the expected answer.\ndoes the number in the provided answer exactly match the expected number?respond with 'yes' or 'no' only.\n\nquestion: what is 2 + 2?", 'mathematical_correct': True}
{'question': 'What is 3 * 3?', 'expected_answer': '9', 'mistral_answer': '9', 'model_judgment': "question: what is 3 * 3?\nexpected answer: 9\nanswer provided by mistral: 9\n\ncompare only the numeric value of the provided answer to the expected answer.\ndoes the number in the provided answer exactly match the expected number?respond with 'yes' or 'no' only.\n\nquestion: what is 3 * 3?", 'mathematical_correct': True}
{'question': 'What is 10 - 3?', 'expected_answer': '7', 'mistral_answer': '6', 'model_judgment': "question: what is 10 - 3?\nexpected answer: 7\nanswer provid

### google/flan-t5-base

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the FLAN-T5 model and tokenizer
model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example dataset
dataset = [
    {"question": "What is 2 + 2?", "expected_answer": "4", "mistral_answer": "The answer is 4"},
    {"question": "What is 3 * 3?", "expected_answer": "9", "mistral_answer": "When multiplying, 3 * 3 equals 9"},
    {"question": "What is 10 - 3?", "expected_answer": "7", "mistral_answer": "Subtracting, we get 6"}
]

# Function to evaluate correctness
def evaluate_correctness(question, mistral_answer, expected_answer):
    # Construct the prompt
    prompt = (
        f"Question: {question}\n"
        f"Expected answer: {expected_answer}\n"
        f"Answer provided by Mistral: {mistral_answer}\n\n"
        "Does the provided answer mean the same as the expected answer? "
        "Respond with 'Yes' or 'No' only."
    )
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    
    # Generate the output
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=20,  # Generate only up to 20 new tokens
        num_return_sequences=1
    )
    
    # Decode the model's response
    model_response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
    
    # Determine correctness
    mathematical_correct = "yes" in model_response
    
    return {
        "model_judgment": model_response,
        "mathematical_correct": mathematical_correct
    }

# Evaluate the dataset
results = []
for item in dataset:
    result = evaluate_correctness(
        question=item["question"],
        mistral_answer=item["mistral_answer"],
        expected_answer=item["expected_answer"]
    )
    results.append({**item, **result})

# Print the results
for res in results:
    print(res)


{'question': 'What is 2 + 2?', 'expected_answer': '4', 'mistral_answer': 'The answer is 4', 'model_judgment': 'no', 'mathematical_correct': False}
{'question': 'What is 3 * 3?', 'expected_answer': '9', 'mistral_answer': 'When multiplying, 3 * 3 equals 9', 'model_judgment': 'yes', 'mathematical_correct': True}
{'question': 'What is 10 - 3?', 'expected_answer': '7', 'mistral_answer': 'Subtracting, we get 6', 'model_judgment': 'no', 'mathematical_correct': False}


In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the FLAN-T5 model and tokenizer
model_name = "google/flan-t5-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example dataset
dataset = [
    {"question": "What is 2 + 2?", "expected_answer": "4", "mistral_answer": "The answer is 4", "evaluation": False},
    {"question": "What is 3 * 3?", "expected_answer": "9", "mistral_answer": "When multiplying, 3 * 3 equals 9", "evaluation": True},
    {"question": "What is 10 - 3?", "expected_answer": "7", "mistral_answer": "Subtracting, we get 6", "evaluation": False}
]

# Function to evaluate correctness of the evaluation
def evaluate_correctness(question, mistral_answer, expected_answer, evaluation):
    # Construct the prompt
    prompt = (
        f"Question: {question}\n"
        f"Expected answer: {expected_answer}\n"
        f"Answer provided by Mistral: {mistral_answer}\n"
        f"Evaluation says correctness is: {evaluation}\n\n"
        "Is this evaluation correct based on the Mistral answer and expected answer? "
        "Respond with 'Yes' or 'No' only."
    )
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    
    # Generate the output
    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=20,  # Generate only up to 20 new tokens
        num_return_sequences=1
    )
    
    # Decode the model's response
    model_response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip().lower()
    
    # Determine correctness
    evaluation_correct = "yes" in model_response
    
    return {
        "model_judgment": model_response,
        "evaluation_correct": evaluation_correct
    }

# Evaluate the dataset
results = []
for item in dataset:
    result = evaluate_correctness(
        question=item["question"],
        mistral_answer=item["mistral_answer"],
        expected_answer=item["expected_answer"],
        evaluation=item["evaluation"]
    )
    results.append({**item, **result})

# Print the results
for res in results:
    print(res)


{'question': 'What is 2 + 2?', 'expected_answer': '4', 'mistral_answer': 'The answer is 4', 'evaluation': False, 'model_judgment': 'no', 'evaluation_correct': False}
{'question': 'What is 3 * 3?', 'expected_answer': '9', 'mistral_answer': 'When multiplying, 3 * 3 equals 9', 'evaluation': True, 'model_judgment': 'yes', 'evaluation_correct': True}
{'question': 'What is 10 - 3?', 'expected_answer': '7', 'mistral_answer': 'Subtracting, we get 6', 'evaluation': False, 'model_judgment': 'no', 'evaluation_correct': False}


## Exact match comparison

After trying different approches I came to a conclusion that in this case when I have a test data set. Which I don't know if it's good because it might be that, or most likely, that Mistrals Large model has been trained on it. But it is too time consuming to create a data set on my own. 

So I will extract the final numerical answer from the Mistral model and then compare them to the correct answers in the test data set. Based on that I can get some statistical measurments. 

Usage of smaller open source LLMs proved to be to unrealible compared to this approach. 

In [39]:
import os
import json
from dotenv import load_dotenv
from mistralai import Mistral

# Load environment variables
load_dotenv()

# Get API key and agent ID
api_key = os.environ["MISTRAL_API_KEY"]
agent_id = os.environ["MISTRAL_AGENT_ID"]

# Initialize Mistral client
client = Mistral(api_key=api_key)

# Load questions from the JSON file
input_file = "sample_questions.json"  
output_file = "model_answers.json"  # Output file name

with open(input_file, "r") as f:
    questions = json.load(f)

# Initialize a list to store the results
results = []

# Iterate over the questions
for question in questions:
    try:
        # Ask the question to the model
        chat_response = client.agents.complete(
            agent_id=agent_id,
            messages=[
                {
                    "role": "user",
                    "content": question
                },
            ],
        )
        # Extract the model's answer
        answer = chat_response.choices[0].message.content.strip()

        # Append the question and answer to the results
        results.append({
            "question": question,
            "answer": answer
        })

        #print(f"Question: {question}")
        #print(f"Answer: {answer}\n")
    except Exception as e:
        print(f"Error processing question: {question}")
        print(f"Error: {e}")

# Save the results to the output file
with open(output_file, "w") as f:
    json.dump(results, f, indent=4)

#print(f"Results saved to {output_file}")


One of the main problems is the format of the answer. When humans answer math questions we use logic and know that there are several ways to write the same thing. But when an AI answers the question it doesn't use logic, it is a probability machine. So, how can we get the right part from the out put extraceted, so that we don't need to evaluate the extraction as well. And, when we have answerd math problems it is never enough with the answer, we also need the resoning. How can we validate the reasoning of the answer. There are cases where the final answer is right but the reasoning is wrong? Like in a human case, you use the calculator to answer and guess how you come to the answer. 

In [38]:
import json
import re  # Import re for regular expressions

# Load the dataset
with open("model_answers.json", "r") as file:
    data = json.load(file)

# Function to extract the final numerical answer
def extract_final_answer(entry):
    answer_text = entry["answer"]
    # Use a regular expression to extract the number after "The answer is"
    match = match = re.search(r"(?:The answer is)\s*(.*)", answer_text)
    if match:
        # Extract and return the number as a float
        final_answer = match.group(1)
        return {"question": entry["question"], "answer": final_answer}
    else:
        # If no match is found, return "Answer not found"
        return {"question": entry["question"], "answer": "Answer not found"}

# Process each entry in the dataset
processed_data = [extract_final_answer(entry) for entry in data]

# Save the processed data to a new JSON file
with open("model_final_answers.json", "w") as file:
    json.dump(processed_data, file, indent=4)

print("Processed data saved to model_final_answers.json")

Processed data saved to model_final_answers.json


In [28]:
import json

# Load the two JSON files
with open("model_final_answers.json", "r") as model, open("sample_clean_qa_data.json", "r") as correct:
    data1 = json.load(model)
    data2 = json.load(correct)

# Create dictionaries for faster comparison
answers1 = {entry["question"]: entry["answer"] for entry in data1}
answers2 = {entry["question"]: entry["answer"] for entry in data2}

# Compare answers for the same questions
results = []
for question in answers1:
    if question in answers2:
        match = answers1[question] == answers2[question]
        results.append({
            "question": question,
            "answer_model": answers1[question],
            "answer_correct": answers2[question],
            "match": match
        })
    else:
        results.append({
            "question": question,
            "answer_model": answers1[question],
            "answer_correct": "Not found in file2",
            "match": False
        })

# Check for questions in file2 but not in file1
for question in answers2:
    if question not in answers1:
        results.append({
            "question": question,
            "answer_model": "Not found in file1",
            "answer_correct": answers2[question],
            "match": False
        })

# Save the results to a new JSON file
with open("comparison_results.json", "w") as result_file:
    json.dump(results, result_file, indent=4)

print("Comparison complete. Results saved to comparison_results.json")

Comparison complete. Results saved to comparison_results.json
